In [1]:
Corpus="./data/train/MultiTrain.Shuffled.csv"
Testset="./data/dev/MultiDev.csv"


In [2]:
from keras.preprocessing import text, sequence
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
import text_normalization as tn
import gensim
import numpy as np
import pandas as pd
import io

def LoadData(Corpus,ClassesDict,Arabic=False): ## loading file
    DF=pd.read_csv(Corpus,converters={'text': str})
    labels=[ClassesDict[x] for x in DF['label'].tolist()]
    sentences=DF['text'].tolist()
    if Arabic:
        
        sentences=[tn.NormForWord2Vec(line) for line in sentences]
    TrueLabels=labels
    labels=to_categorical(np.asarray(labels), num_classes=len(ClassesDict))
    return sentences,labels,TrueLabels


Using TensorFlow backend.
C:\Users\dell\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\dell\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\dell\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\dell\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In [3]:
import pandas as pd
import numpy as np
import re
import os

import pandas as pd
import numpy as np
import re
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer

!pip install Tashaphyne
import pyarabic.arabrepr
from tashaphyne.stemming import ArabicLightStemmer

import random
from sklearn.model_selection import train_test_split


In [4]:
df=pd.read_csv(Corpus)
df

,Unnamed: 0,label,text
0,0,MSA,بالإضافة لقيام معلمو الجيزة للذهاب إلي جريدة ا...
1,1,MSA,بعدين والله حرام تجي تلقى الي واقف عند الاشاره...
2,2,DIAL_LEV,لمسه اليد مرتين واضحة جدا والحكم
3,3,DIAL_LEV,بخصوص الهاتريك عمرها ما راح تصير
4,4,DIAL_GLF,الله يجبر كسرهم ويرجع و لدهم اليوم قبل بكرى ،
...,...,...,...
86536,86536,DIAL_LEV,وبورسعيد حكايتها
86537,86537,MSA,نعم الان بعد لبس الزي الرسمي السعودي يكون لنا ...
86538,86538,DIAL_GLF,اتوقع والله اعلم انه يقصد بقية الشعب المسافرين...
86539,86539,DIAL_EGY,هي بيتبدأ


In [5]:
#yall nrmove 7roof el gr w elstop words... de kol elly l2thom
stop_words =['من',
 'في',
 'على',
 'و',
 'فى',
 'يا',
 'عن',
 'مع',
 'ان',
 'هو',
 'علي',
 'ما',
 'اللي',
 'كل',
 'بعد',
 'ده',
 'اليوم',
 'أن',
 'يوم',
 'انا',
 'إلى',
 'كان',
 'ايه',
 'اللى',
 'الى',
 'دي',
 'بين',
 'انت',
 'أنا',
 'حتى',
 'لما',
 'فيه',
 'هذا',
 'واحد',
 'احنا',
 'اي',
 'كده',
 'إن',
 'او',
 'أو',
 'عليه',
 'ف',
 'دى',
 'مين',
 'الي',
 'كانت',
 'أمام',
 'زي',
 'يكون',
 'خلال',
 'ع',
 'كنت',
 'هي',
 'فيها',
 'عند',
 'التي',
 'الذي',
 'قال',
 'هذه',
 'قد',
 'انه',
 'ريتويت',
 'بعض',
 'أول',
 'ايه',
 'الان',
 'أي',
 'منذ',
 'عليها',
 'له',
 'ال',
 'تم',
 'ب',
 'دة',
 'عليك',
 'اى',
 'كلها',
 'اللتى',
 'هى',
 'دا',
 'انك',
 'وهو',
 'ومن',
 'منك',
 'نحن',
 'زى',
 'أنت',
 'انهم',
 'معانا',
 'حتي',
 'وانا',
 'عنه',
 'إلي',
 'ونحن',
 'وانت',
 'منكم',
 'وان',
 'معاهم',
 'معايا',
 'وأنا',
 'عنها',
 'إنه',
 'اني',
 'معك',
 'اننا',
 'فيهم',
 'د',
 'انتا',
 'عنك',
 'وهى',
 'معا',
 'آن',
 'انتي',
 'وأنت',
 'وإن',
 'ومع',
 'وعن',
 'معاكم',
 'معاكو',
 'معاها',
 'وعليه',
 'وانتم',
 'وانتي',
 '¿',
 '|']


In [6]:
def normalize(sentence):
    '''
    Argument:
        string of words
    return:
        string of words but standardize the words
    '''
    sentence = re.sub("[إأآا]", "ا", sentence)
    sentence = re.sub("ى", "ي", sentence)
    sentence = re.sub("ؤ", "ء", sentence)
    sentence = re.sub("ئ", "ء", sentence)
    sentence = re.sub("ة", "ه", sentence)
    sentence = re.sub("گ", "ك", sentence)
    return sentence


In [7]:
def removing_ar_stopwords(text):
    """
        Here we remove all Arabic stop words
        
    """
      # if read it from file
#     ar_stopwords_list = open("arabic_stopwords.txt", "r") 
#     stop_words = ar_stopwords_list.read().split("\n")
#     stop_words = []
    original_words = []
    words = word_tokenize(text) # it works on one hadith not list
    for word in words:
        if word not in stop_words:
            original_words.append(word)
    filtered_sentence = " ".join(original_words)
    return filtered_sentence


In [8]:
def clearReg(text):
    """
        This function for getting the normal values of out of lemmatization function
        that takse a string of dict as a 
        takes  : '{"result":["امر","ب","أخذ","ما","نهى","ه","انتهى"]}'
        return : ['امر أخذ ما نهى انتهى']
    """
    each_lemma_word = []
    each_lemma_sentence = []
    for hadith in text:
        matches = re.findall(r'\"(.+?)\"',hadith)
        for word in matches:
            if len(word) >= 2 and word !='result':
                each_lemma_word.append(word)
        each_lemma_sentence.append(" ".join(each_lemma_word))
        each_lemma_word.clear()
    return each_lemma_sentence


In [9]:
def stemming_1(text):
    """
        This is first functoin for stemming and it's looks not good accurac, NLTK by ISRIStemmer.
    """
    st = ISRIStemmer()
    stemmend_words = []
    words = word_tokenize(text)
    for word in words:
        stemmend_words.append(st.stem(word))
    stemmed_sentence = " ".join(stemmend_words)
    return stemmed_sentence
        
    
    
def stemming_2(text):
    """
        This is Second functoin for stemming and it's looks good results, with built in library called Tashaphyne.
        The documentation here ==> https://pypi.org/project/Tashaphyne/
    
    """
    import pyarabic.arabrepr
    arepr = pyarabic.arabrepr.ArabicRepr()
    repr = arepr.repr

    from tashaphyne.stemming import ArabicLightStemmer
    ArListem = ArabicLightStemmer()

    Tweets_without_stop_words_and_with_normalization_and_with_stemming = []

    for tweet in Tweets_without_stop_words_and_with_normalization_and_with_stemming:
        words = word_tokenize(tweet)
        new_list = []
        for word in words:
            stem = ArListem.light_stem(word)
            stem = ArListem.get_stem()
            new_list.append(stem)

        tweets_sentence_with_stemming = " ".join(new_list)
        tweets_without_stop_words_and_with_normalization_and_with_stemming.append(hadith_sentence_with_stemming)
        
    return hadiths_without_stop_words_and_with_normalization_and_with_stemming


In [10]:
def lemmatization(text):
    """
        This function for lemma Arabic words by API, and it getting best result of the previous functions
        return a string dictinary like exactly '{"result":["امر","ب","أخذ","ما","نهى","ه","انتهى"]}'
    """
    import http.client
    conn = http.client.HTTPSConnection("farasa-api.qcri.org")
    hadith_dict = {}
    list_pyload_input = []
    list_pyload_out = []
    length = len(text)
    for h in text[:length]:
        q = '{"text":'+'"{}"'.format(h)+'}'
        list_pyload_input.append(q)
    headers = { 'content-type': "application/json", 'cache-control': "no-cache", }
    for h in list_pyload_input:
        conn.request("POST", "/msa/webapi/lemma", h.encode('utf-8'), headers)
        res = conn.getresponse()
        data = res.read()
        list_pyload_out.append(data.decode("utf-8"))
        final_result = clearReg(list_pyload_out)     # call clearReg for clean the text
    return final_result


Upove some functions for like:

stemming_1 by ISRIStemmer from NLTK.

stemming_2 by Tashaphyne is an Arabic light stemmer(removing prefixes and suffixes) and give all possible segmentations.

lemmatization by Farasa API

By the experimental: lemmatization by Farasa have a good results.



In [11]:
all_tweets_1 = []
all_removedIndex_1 = []
counter = 0

#hna n2s mn 86541 l 85658 3l4an kda h7zn index bta3 elee etms7
for tweet in df['text']:
    counter = counter +1;
    if(isinstance(tweet, str)):
        all_tweets_1.append(tweet)
    else:
        all_removedIndex_1.append(counter-1)

    


In [12]:
len(all_tweets_1)


85658

In [13]:
len(all_removedIndex_1)

883

In [14]:
86541 -85658  #tmaaaaaaam

883

In [15]:
all_removedIndex_1

[18,
 37,
 117,
 242,
 398,
 548,
 609,
 824,
 879,
 884,
 1265,
 1391,
 1490,
 1491,
 1646,
 2018,
 2293,
 2300,
 2462,
 2463,
 2464,
 2477,
 2478,
 2514,
 2606,
 2908,
 3146,
 3285,
 3336,
 3597,
 3683,
 3712,
 3713,
 3714,
 3715,
 3716,
 3717,
 3718,
 3901,
 4532,
 4754,
 4757,
 4842,
 5171,
 5273,
 5274,
 5275,
 5917,
 6000,
 6225,
 6308,
 6425,
 6578,
 6820,
 6856,
 6890,
 6891,
 7169,
 7446,
 7667,
 7681,
 7682,
 7835,
 7975,
 8011,
 8194,
 8337,
 8466,
 8640,
 8721,
 8857,
 9222,
 9255,
 9318,
 10334,
 10566,
 10642,
 10677,
 10699,
 10775,
 10935,
 11155,
 11324,
 11472,
 11768,
 12014,
 12031,
 12383,
 12384,
 12506,
 12518,
 12640,
 12901,
 12902,
 13103,
 13207,
 14005,
 14292,
 14895,
 14933,
 14934,
 14935,
 14936,
 14937,
 14938,
 14939,
 14940,
 14941,
 14942,
 15531,
 15593,
 15704,
 15713,
 16037,
 16129,
 16134,
 16185,
 16202,
 16206,
 16238,
 16730,
 16812,
 17277,
 17319,
 17429,
 17507,
 17575,
 17799,
 18005,
 18046,
 18047,
 18048,
 18049,
 18133,
 18157,
 18372

In [16]:
df['text'][2300]  #wwwwwwwwwwoooow

nan

In [17]:
df['text'].isnull()

0        False
1        False
2        False
3        False
4        False
         ...  
86536    False
86537    False
86538    False
86539    False
86540    False
Name: text, Length: 86541, dtype: bool

In [18]:
df['text'][17]

'كنت أتمنى ان يفرد الكاتب نبذة عن تطور المجتمع التونسي عبر التاريخ والقيم والمثل الرائعة التي يتصفون بها ومواقفهم البطوليه الرائعة والملحمة الإسلامية التي سطروها عبر'

In [19]:
df.dropna(inplace=True, how='all')


ya 5rabyyyyyyyy

yalaaa n3ml el pre processing 3l data

In [20]:
%%time

# preprocess the tweets
cleared_tweet_1 = []           # Removing stopwords
cleared_tweet_1_2 = []         # Normalization
cleared_tweet_1_2_3 = []       # Lematization

for tweet in all_tweets_1[0:10]:
    cleared_tweet_1.append(removing_ar_stopwords(tweet))         # Removing stopwords
for tweets in cleared_tweet_1:
    cleared_tweet_1_2.append(normalize(tweets))                   # Normalization
cleared_tweet_1_2_3 = lemmatization(cleared_tweet_1_2)           # Lematization

print('The size of data:')
len(cleared_tweet_1), len(cleared_tweet_1_2), len(cleared_tweet_1_2_3)


The size of data:
Wall time: 1.53 s


(10, 10, 10)

In [21]:
cleared_tweet_1_2_3

['اضافه قيام معلمو جيزه ذهاب جريد سابع نشر خبر اتي عام',
 'بعد الله حرام تجي تلقي واقف اشاره عرض متعدي ناس خط مشاه مصيبه واقف قدام رجل مرور لكن لا كان',
 'لمس يد مرة واضح جد حكم',
 'خصوص هاتريك عمر راح صار',
 'الله أجبر كسر رجع أدهم قبل بكر',
 'تعدد اسباب موت حد عمر وث اوراق شجرة لبد سقط يابن ادم انما ايام اذ ذهب بعض ذهب يوم كما حسن بصري رضي الله وبديعاقب تسبب حادث ارواح ناس ماهي لعب اوعلبه متي انتهى من',
 'الله يزرقكم واسع',
 'ملفت مبهر ما احتاج تامل سبع',
 'سلام على رحم',
 'اغلب تعليق سابق اقل وصف انها سخيف وبه افتراء']

In [22]:
df1=pd.read_csv("Arabic_Data.csv")

In [23]:
df2 = pd.read_csv("Arabic_Data2.csv")

In [24]:
df3= pd.read_csv("Arabic_Data3.csv")

In [25]:
df4 = pd.read_csv("Arabic_Data4.csv")

In [26]:
df5 = pd.read_csv("Arabic_Data5.csv")

In [123]:
6*4

24

In [31]:
df1['Arabic_Tweets_Cleaned'][0]

'اضافه قيام معلمو جيزه ذهاب جريد سابع نشر خبر اتي عام'

In [29]:
df2['Arabic_Tweets_Cleaned'][0]

'بركه زار خدم مدنيه زار تربيه تعليم خريج خريج خدم اجتماعيه لا وظيف اوموظفين مسمى وظيفي اخري واحد من'

In [32]:
df[0:12001]

,Unnamed: 0,label,text
0,0,MSA,بالإضافة لقيام معلمو الجيزة للذهاب إلي جريدة ا...
1,1,MSA,بعدين والله حرام تجي تلقى الي واقف عند الاشاره...
2,2,DIAL_LEV,لمسه اليد مرتين واضحة جدا والحكم
3,3,DIAL_LEV,بخصوص الهاتريك عمرها ما راح تصير
4,4,DIAL_GLF,الله يجبر كسرهم ويرجع و لدهم اليوم قبل بكرى ،
...,...,...,...
11996,11996,MSA,هل عرفتم الحقيقه يا جماهيرنا
11997,11997,MSA,هذه من مساوء التعداد . . !
11998,11998,MSA,ومصدره واحد يأكله ليسد جوعه ليبقى
11999,11999,MSA,{ إِنَّ أَكْرَمَكُمْ عِندَ اللَّهِ أَتْقَاكُمْ


In [118]:
#all_removedIndex_1   doll b null

[18,
 37,
 117,
 242,
 398,
 548,
 609,
 824,
 879,
 884,
 1265,
 1391,
 1490,
 1491,
 1646,
 2018,
 2293,
 2300,
 2462,
 2463,
 2464,
 2477,
 2478,
 2514,
 2606,
 2908,
 3146,
 3285,
 3336,
 3597,
 3683,
 3712,
 3713,
 3714,
 3715,
 3716,
 3717,
 3718,
 3901,
 4532,
 4754,
 4757,
 4842,
 5171,
 5273,
 5274,
 5275,
 5917,
 6000,
 6225,
 6308,
 6425,
 6578,
 6820,
 6856,
 6890,
 6891,
 7169,
 7446,
 7667,
 7681,
 7682,
 7835,
 7975,
 8011,
 8194,
 8337,
 8466,
 8640,
 8721,
 8857,
 9222,
 9255,
 9318,
 10334,
 10566,
 10642,
 10677,
 10699,
 10775,
 10935,
 11155,
 11324,
 11472,
 11768,
 12014,
 12031,
 12383,
 12384,
 12506,
 12518,
 12640,
 12901,
 12902,
 13103,
 13207,
 14005,
 14292,
 14895,
 14933,
 14934,
 14935,
 14936,
 14937,
 14938,
 14939,
 14940,
 14941,
 14942,
 15531,
 15593,
 15704,
 15713,
 16037,
 16129,
 16134,
 16185,
 16202,
 16206,
 16238,
 16730,
 16812,
 17277,
 17319,
 17429,
 17507,
 17575,
 17799,
 18005,
 18046,
 18047,
 18048,
 18049,
 18133,
 18157,
 18372

In [27]:
#counter2 = -1
#index1= 0
#index2=0
#for i in df['text']:
#    counter2 = counter2 +1
#    index2 = index2+1
#    if counter2 not in all_removedIndex_1:
#        df_clean['text'][index1] = df['text'][index2]
#        index1 = index+ 1
